# Слепо пребарување

## Референци

1. [Пребарување прво по широчина](https://en.wikipedia.org/wiki/Breadth-first_search)
1. [Пребарување прво по длабочина](https://en.wikipedia.org/wiki/Depth-first_search)
1. [Разлика меѓу листа, торка, множество и речник](https://thomas-cokelaer.info/tutorials/python/data_structures.html)

In [17]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('jiEb-9IYPs0')

![](images/graph.png)

## Претставување граф преку речник

In [1]:
class Graph():
    def __init__(self):
        """
        Initialises an empty dict as the graph data structure.
        """
        self.graph_dict = {}
    
    def add_vertex(self, vertex):
        """
        Adds a vertex to the graph.
        
        Args:
            vertex: vertex to be added in the graph
        """
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = []
    
    def vertices(self):
        """
        Returns the graph's vertices.
        """
        return list(self.graph_dict.keys())
    
    def add_edge(self, edge, add_reversed=True):
        """
        Adds an edge to the graph.
        
        Args:
            edge: a tupple of two vertices, (first_vertex, second_vertex)
            add_reversed: whether to add the edge in reversed direction, (second_vertex, first_vertex)
        """
        vertex1, vertex2 = edge
        self.graph_dict[vertex1].append(vertex2)
        if add_reversed:
            self.graph_dict[vertex2].append(vertex1)
    
    def edges(self):
        """
        Returns a list of all edges in the graph.
        """
        edges = []
        for vertex in self.graph_dict:
            for neighbour in self.graph_dict[vertex]:
                edges.append((vertex, neighbour))
        return edges
    
    def neighbours(self, vertex):
        """
        Returns all neighbours of the given vertex.
        """
        return self.graph_dict[vertex]
    
    def remove_vertex(self, vertex_to_remove):
        """
        Removes a vertex from the graph.
        
        First, the vertex's list is removed.
        Then, we remove all the occurances of the vertex in another vertex's list.
        
        Args:
            vertex_to_remove: the vertex to be removed.
        """
        del self.graph_dict[vertex_to_remove]
        for vertex in self.vertices():
            if vertex_to_remove in self.graph_dict[vertex]:
                self.graph_dict[vertex].remove(vertex_to_remove)

    def remove_edge(self, edge_to_remove, remove_reversed=True):
        """
        Removes an edge from the graph.
        
        Args:
            edge_to_remove: the edge to be removed
            remove_reversed: whether to remove the edge in reversed direction
        """
        vertex1, vertex2 = edge_to_remove
        if vertex2 in self.graph_dict[vertex1]:
            self.graph_dict[vertex1].remove(vertex2)
        if remove_reversed:
            if vertex1 in self.graph_dict[vertex2]:
                self.graph_dict[vertex2].remove(vertex1)

    def isolated_vertices(self):
        """
        Returns a list of all isolated vertices.
        """
        isolated_vertices = []
        for vertex in self.graph_dict:
            if not self.graph_dict[vertex]:
                isolated_vertices.append(vertex)
        return isolated_vertices

In [2]:
g = Graph()

In [3]:
g.add_vertex('S')
g.add_vertex('A')
g.add_vertex('B')
g.add_vertex('C')
g.add_vertex('D')
g.add_vertex('E')
g.add_vertex('F')
g.add_vertex('G')
g.add_vertex('H')

In [4]:
g.vertices()

['S', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [5]:
g.add_edge(('S', 'A'))
g.add_edge(('S', 'C'))
g.add_edge(('S', 'G'))
g.add_edge(('A', 'B'))
g.add_edge(('C', 'D'))
g.add_edge(('C', 'E'))
g.add_edge(('C', 'F'))
g.add_edge(('G', 'F'))
g.add_edge(('G', 'H'))
g.add_edge(('E', 'H'))

In [6]:
g.edges()

[('S', 'A'),
 ('S', 'C'),
 ('S', 'G'),
 ('A', 'S'),
 ('A', 'B'),
 ('B', 'A'),
 ('C', 'S'),
 ('C', 'D'),
 ('C', 'E'),
 ('C', 'F'),
 ('D', 'C'),
 ('E', 'C'),
 ('E', 'H'),
 ('F', 'C'),
 ('F', 'G'),
 ('G', 'S'),
 ('G', 'F'),
 ('G', 'H'),
 ('H', 'G'),
 ('H', 'E')]

In [7]:
g.neighbours('S')

['A', 'C', 'G']

## Пребарување прво по широчина

In [8]:
from collections import deque

In [9]:
def breadth_first_search_find_path(graph, starting_vertex, goal_vertex, verbose=False):
    """
    Returns the path from starting_vertex to goal_vertex using the DFS algorithm.
    """
    # Ако почетниот јазол е еднаков на целниот, тогаш нема логика да пребаруваме воопшто
    if starting_vertex == goal_vertex:
        if verbose:
            print('Почетниот и бараниот јазол се исти')
        return []
    # Користиме листа на посетени јазли која всушност е податочна структура множество. 
    # За посетен јазол го сметаме оној јазол кој ќе го истражиме како сосед на јазолот кој го разгрануваме.
    visited = {starting_vertex}
    # Користиме двојно поврзана листа која ни е редицата од која ќе го земаме следниот јазол за разгранување.
    # Тука ја памтиме и моменталната патека за секој јазол од почетниот.
    queue = deque([[starting_vertex]])
    # Пребаруваме сѐ додека има јазли за разгранување во редицата.
    while queue:
        if verbose:
            print('Ред за разгранување:')
            for element in queue:
                print(element, end=' ')
            print()
            print()
        # Членови на редицата јазли се патеките од почетниот јазол до некој јазол кој треба да се разграни.
        # За да го земаме наредниот јазол за разгранување, 
        #    ние треба од редицата да ја извадиме патеката на тој јазол.
        vertex_list = queue.popleft()
        # Јазолот за разгранување е послениот во оваа листа.
        vertex_to_expand = vertex_list[-1]
        if verbose:
            print('Го разгрануваме јазолот {}'.format(vertex_to_expand))
        # Го разгрануваме така што пребаруваме низ сите негови соседи.
        for neighbour in graph.neighbours(vertex_to_expand):
            if neighbour in visited:
                if verbose:
                    print('{} е веќе посетен'.format(neighbour))
            else:
                # Ако некој сосед не е посетен, тогаш го додаваме во листата на посетени, 
                #     и во редицата на јазли за разгранување.
                if verbose:
                    print('{}, кој е соседен јазол на {} го немаме посетено до сега, затоа го додаваме во редот '
                          'за разгранување и го означуваме како посетен'.format(neighbour, vertex_to_expand))
                # Тука ја вршиме проверката дали сме стигнале до целниот јазол 
                if neighbour == goal_vertex:
                    if verbose:
                        print('Го пронајдовме посакуваниот јазол {}. Патеката да стигнеме до тука е {}'
                              .format(neighbour, vertex_list + [neighbour]))
                    return vertex_list + [neighbour]
                visited.add(neighbour)
                # Бидејќи ова е пребарување прво по широчина, 
                #     соседот го додаваме на крајот од редицата јазли за разгранување.
                # Соседот го врзуваме со патеката од почетниот јазол до моменталниот кој го разгрануваме.
                queue.append(vertex_list + [neighbour])
        if verbose:
            print()

In [10]:
breadth_first_search_find_path(g, 'S', 'E', verbose=True)

Ред за разгранување:
['S'] 

Го разгрануваме јазолот S
A, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
C, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
G, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен

Ред за разгранување:
['S', 'A'] ['S', 'C'] ['S', 'G'] 

Го разгрануваме јазолот A
S е веќе посетен
B, кој е соседен јазол на A го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен

Ред за разгранување:
['S', 'C'] ['S', 'G'] ['S', 'A', 'B'] 

Го разгрануваме јазолот C
S е веќе посетен
D, кој е соседен јазол на C го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
E, кој е соседен јазол на C го немаме посетено до сега, затоа го додаваме во редот за разгранување и

['S', 'C', 'E']

## Пребарување прво по длабочина

In [11]:
def depth_first_search_find_path(graph, starting_vertex, goal_vertex, verbose=False):
    """
    Returns the path from starting_vertex to goal_vertex using the BFS algorithm.
    """
    # Ако почетниот јазол е еднаков на целниот, тогаш нема логика да пребаруваме воопшто
    if starting_vertex == goal_vertex:
        if verbose:
            print('Почетниот и бараниот јазол се исти')
        return []
    # Користиме листа на посетени јазли која всушност е податочна структура множество. 
    # За посетен јазол го сметаме оној јазол кој ќе го истражиме како сосед на јазолот кој го разгрануваме.
    visited = {starting_vertex}
    # Користиме двојно поврзана листа која ни е редицата од која ќе го земаме следниот јазол за разгранување.
    # Тука ја памтиме и моменталната патека за секој јазол од почетниот.
    queue = deque([[starting_vertex]])
    # Пребаруваме сѐ додека има јазли за разгранување во редицата.
    while queue:
        if verbose:
            print('Ред за разгранување:')
            for element in queue:
                print(element, end=' ')
            print()
            print()
        # Членови на редицата јазли се патеките од почетниот јазол до некој јазол кој треба да се разграни.
        # За да го земаме наредниот јазол за разгранување, 
        #    ние треба од редицата да ја извадиме патеката на тој јазол.
        vertex_list = queue.popleft()
        # Јазолот за разгранување е послениот во оваа листа.
        vertex_to_expand = vertex_list[-1]
        if verbose:
            print('Го разгрануваме јазолот {}'.format(vertex_to_expand))
        # Го разгрануваме така што пребаруваме низ сите негови соседи.
        for neighbour in graph.neighbours(vertex_to_expand):
            if neighbour in visited:
                if verbose:
                    print('{} е веќе посетен'.format(neighbour))
            else:
                # Ако некој сосед не е посетен, тогаш го додаваме во листата на посетени, 
                #     и во редицата на јазли за разгранување.
                if verbose:
                    print('{}, кој е соседен јазол на {} го немаме посетено до сега, затоа го додаваме во редот '
                          'за разгранување и го означуваме како посетен'.format(neighbour, vertex_to_expand))
                # Тука ја вршиме проверката дали сме стигнале до целниот јазол 
                if neighbour == goal_vertex:
                    if verbose:
                        print('Го пронајдовме посакуваниот јазол {}. Патеката да стигнеме до тука е {}'
                              .format(neighbour, vertex_list + [neighbour]))
                    return vertex_list + [neighbour]
                visited.add(neighbour)
                # Бидејќи ова е пребарување прво по длабочина, 
                #     соседот го додаваме на почетокот од редицата јазли за разгранување.
                # Соседот го врзуваме со патеката од почетниот јазол до моменталниот кој го разгрануваме.
                queue.appendleft(vertex_list + [neighbour])
        if verbose:
            print()

In [12]:
depth_first_search_find_path(g, 'S', 'E', verbose=True)

Ред за разгранување:
['S'] 

Го разгрануваме јазолот S
A, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
C, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
G, кој е соседен јазол на S го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен

Ред за разгранување:
['S', 'G'] ['S', 'C'] ['S', 'A'] 

Го разгрануваме јазолот G
S е веќе посетен
F, кој е соседен јазол на G го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен
H, кој е соседен јазол на G го немаме посетено до сега, затоа го додаваме во редот за разгранување и го означуваме како посетен

Ред за разгранување:
['S', 'G', 'H'] ['S', 'G', 'F'] ['S', 'C'] ['S', 'A'] 

Го разгрануваме јазолот H
G е веќе посетен
E, кој е соседен јазол на H го немаме посетено до сега, затоа го додаваме во редот з

['S', 'G', 'H', 'E']